In [ ]:
#--------- ANALYSIS SCRIPT---------------

# This script searches the LAMMPS_simulations directory and 
# 1. Identifies simulations that have not yet been analysed
# 2. Creates ovito files and renderings
# 3. Runs the following analyses as functions of density or cooling rate:
#   a) % of sp, sp2, sp3 environments 
#   c) Histogram of ring sizes for a given density
#   d) Frequency plots for n-mem rings
#   e) Radial Distribution Functions
#   f) Potential Energy
#   g) Bond Length

# # -------- OPTIONAL WAIT FUNCTION TO ALLOW FOR AUTOMATED RUNNING ---------
# import subprocess
# import time
# WAIT_TIME = 600  # seconds
# USER = "scat9451"

# while True:
#     job_status = subprocess.run(["qstat", "-u", USER], capture_output=True, text=True)
#     if not job_status.stdout.strip():  # empty means no jobs
#         break
#     print("Active jobs found - waiting ...")
#     time.sleep(WAIT_TIME)

# print("No active jobs found - proceeding ...")
# # ---------------------------------------------------------------------

import re
import numpy as np
import ovito
from ovito.io import import_file
from ovito.modifiers import CreateBondsModifier, FindRingsModifier, CoordinationAnalysisModifier, ColorCodingModifier, BondAnalysisModifier
from ovito.vis import Viewport, TachyonRenderer, ColorLegendOverlay, BondsVis
from ovito.qt_compat import QtCore

# ------ MAKE NEW DIRECTORIES ------
from pathlib import Path
import re

cwd = Path.cwd()

analysis_dir = cwd / "Analysis"
analysis_dir.mkdir(exist_ok=True)

ovito_dir = analysis_dir / "Ovito"
ovito_dir.mkdir(exist_ok=True)

structural_analysis_dir = analysis_dir / "Structural Analysis"
structural_analysis_dir.mkdir(exist_ok=True)
# ----------------------------------

# ------  IMPORT SIMULATION DATA ------
# Searches through the specified directory and:
# Returns "datafiles" which is a list of 2 element tuples; 
# 1 is the run dir name (containing all MD run information) (str) - This is used to generate corresponding file names
# 2 is a list of dump file paths (Path objects)
# 
# e.g. 
# datafiles = [
#   ( "run1", [Path(".../dump_custom.C.0.dat"), Path(".../dump_custom.C.1.dat"), …] ),
#   ( "run2", [Path(".../dump_custom.C.0.dat"), Path(".../dump_custom.C.1.dat"), …] ),
#   …
# ]
# 
# Ensure that the directory structure matches:
# <directory>                                  # Base directory passed to import_simulation_data()
#   ├── <run_...>/                             # Directory containing all repeats of  a given set of params
#   │     ├── <run_1_...>/                     # Second level: each individual MD repeat 
#   │     │     └── NVT/                       # Inside each run directory a folder named “NVT” (generated by LAMMPS)
#   │     │           ├── dump_custom.C.0.dat  # List of dump files
#   │     │           ├── dump_custom.C.1.dat
#   │     │           └── …                
#   │     └── <run_2_...>/
#   │           └── NVT/
#   │                ├── dump_custom.C.0.dat
#   │                └── …
#   └── <run...>/
#         ├── <run_1_...>/
#         │     └── NVT/
#         │        ├── dump_custom.C.0.dat
#         │        └── …
#         └── …


def import_simulation_data(directory):
    directory = Path(directory)
    datafiles = []
    dump_file_name = re.compile(r"^dump_custom\.C\.(\d+)\.dat$")

    for simulation_dir in directory.iterdir():  # For each subdirectory in LAMMPS_simulations
        if not simulation_dir.is_dir():
            print(f"ERROR: {simulation_dir} is not a directory")
            continue

        for run_dir in simulation_dir.iterdir():  # For each individual MD run within each subdirectory
            if not run_dir.is_dir():
                print(f"ERROR: {run_dir} is not a directory")
                continue

            # Import all data dumps in numerical order
            matches = []
            nvt_dir = run_dir / "NVT"
            
            if not nvt_dir.is_dir(): # checks that there aren't any files named "NVT"
                print(f"ERROR: {nvt_dir} is not a directory")
                continue

            for fname in nvt_dir.iterdir():
                
                if not fname.is_file(): # checks that everything inside NVT is a file
                    print(f"ERROR: {fname} is not a file")
                    continue

                m = dump_file_name.match(fname.name)
                if m:
                    num = int(m.group(1))
                    matches.append((num, fname))
            
            if not matches: # Notifies user if there are no dump files in an NVT folder
                print(f"No dump files found in {run_dir.name}")
                continue

            matches.sort(key=lambda x: x[0])
            sorted_files = [p for _, p in matches]
            datafiles.append((run_dir.name, sorted_files))

    if not datafiles:
        print("No LAMMPS datafiles found")
        return

    return datafiles

datafiles = import_simulation_data("LAMMPS_simulations")
print(f"{len(datafiles)} LAMMPS simulation files imported")

# Sets up an empty pipeline for each successive function to use
def empty_ovito_pipeline(datafiles):

    # Clear existing pipeline
    for p in list(ovito.scene.pipelines):
        p.remove_from_scene()

    if not datafiles:
        raise ValueError("No datafiles provided to basic_ovito_pipeline()")
   
    basename, files = datafiles[0]
    first_file = files[0]
    pipeline = import_file(first_file)
    
    return pipeline
pipeline = empty_ovito_pipeline(datafiles)

# Data visualisation in Ovito
def ovito_analysis(datafiles, pipeline):

    if not datafiles:
        raise ValueError("No datafiles provided to ovito_analysis()")

    # ------- ANALYSIS OF IMPORTED FILES ------------
    # BUG: Image and video renderers error with: 
    # "RuntimeError: Visual element 'Rings' reported an error:Failed to build non-periodic representation of periodic surface mesh. Periodic domain might be too small." if ring mod is included.

    # Bond Modifier and Visuals 
    bond_modifier = CreateBondsModifier(cutoff=1.85)
    bond_modifier.vis.width = 0.15
    bond_modifier.vis.coloring_mode = BondsVis.ColoringMode.Uniform
    bond_modifier.vis.color = (0.5, 0.5, 0.5)
    pipeline.modifiers.append(bond_modifier)

    # Coordination Modifier and Colour Coding
    pipeline.modifiers.append(CoordinationAnalysisModifier(cutoff=1.85))
    colour_coding_mod = ColorCodingModifier(property="Coordination",start_value=1.0,end_value=4.0,gradient=ColorCodingModifier.Viridis(),discretize_color_map=True)
    pipeline.modifiers.append(colour_coding_mod)

    # Add to Scene
    pipeline.add_to_scene()

    # Viewing settings
    vp = Viewport()
    vp.type = Viewport.Type.Perspective

    # Coordination Legend
    legend = ColorLegendOverlay(
        title = "Coordination",
        modifier = colour_coding_mod,
        alignment = QtCore.Qt.AlignmentFlag.AlignHCenter | QtCore.Qt.AlignmentFlag.AlignBottom,
        orientation = QtCore.Qt.Orientation.Horizontal,
        font_size = 0.1,
        format_string = '%.0f' 
        )
    vp.overlays.append(legend)

    # Note: this function only renders for the first repeat 
    def is_run_1_(run_file_name):
        return re.match(r"^run_1_C", run_file_name) is not None

    # Skipped file counters
    skipped_ovito_files = 0
    skipped_png_files = 0
    skipped_avi_files = 0
    
    # Image and Video Rendering
    for basename, files in datafiles:

        pipeline.source.load(files)

        if is_run_1_(basename): # Only does analysis for run_1_
            
            ovito_save_file = ovito_dir / f"{basename}.ovito"
            img_save_file   = ovito_dir / f"{basename}.png"
            vid_save_file   = ovito_dir / f"{basename}.avi"                                     

            # Ovito File Existance-Checker
            ovito_exists = any(ovito_dir.rglob(ovito_save_file.name))
            img_exists   = any(ovito_dir.rglob(img_save_file.name))
            vid_exists   = any(ovito_dir.rglob(vid_save_file.name))
            
            # Set particle scaling (datafile specific)
            n_frames = pipeline.source.num_frames
            final_frame = max(0, n_frames - 1)
            data = pipeline.compute(frame = final_frame)
            data.particles.vis.scaling = 0.3

            # Set Zoom
            vp.zoom_all()

            # Save .ovito scene
            if not ovito_exists:
                ovito.scene.save(ovito_save_file)
                print(f"Created: {basename}.ovito") 
            else: skipped_ovito_files +=1

            # Image and animation Render 
            tachyon = TachyonRenderer(shadows=False, direct_light_intensity=1.1)
            if not img_exists:
                vp.render_image(size=(1920,1080),
                                filename=img_save_file,
                                background=(1,1,1),
                                frame=final_frame,
                                renderer=tachyon)
                print(f"Created: {basename}.png") 
            else: skipped_png_files += 1


            if not vid_exists:        
                vp.render_anim(size=(1920,1080), 
                            filename=vid_save_file, 
                            fps=10,
                            renderer=tachyon)
                print(f"Created {basename}.avi")   
            else: skipped_avi_files += 1

    # Print Skipped files
    if skipped_ovito_files:
        print(f"Skipped {skipped_ovito_files} existing .ovito files")
    if skipped_png_files:
        print(f"Skipped {skipped_png_files} existing .png files")
    if skipped_avi_files:
        print(f"Skipped {skipped_avi_files} existing .avi files")
    
    # Remove modifiers
    pipeline.modifiers.pop()
    pipeline.modifiers.pop()
    pipeline.modifiers.pop()


# ------ DATA GENERATION FUNCTIONS ------
# file_analysis(): 
#   1. uses datafiles from import_simulation_data()
#   2. uses the pipeline from empty_ovito_pipeline(): no modifiers by default
#   3. checks if files already exist in "Structural Analysis"
#   4. loads each file in datafiles into the existing pipeline
#   5. computes a specified data object for the given pipeline on each file and saves to a file name given by "unique_file_identifier" and the run-file name
#   NOTE:
#       a) requires "unique_file_identifier": e.g. "bond_length_data" or "RDF" (don't include leading underscore or .txt)
#       b) "data_function" refers to the ovito function that return the desired data object 
#               e.g. "data.particles['Coordination']" or "data.tables['coordination-rdf'].xy()" or "data.particles["c_pea"]" 
#       c) requires use of the "lambda data:" syntax for creating a throwaway function
#               e.g. When calling this func, use "file_analysis_and_existance_checker(datafiles,"ring_data",lambda data: data.tables["ring-size-histogram"].xy())""

def file_analysis(datafiles, pipeline, unique_file_identifier, data_function):

    if not datafiles:
        raise ValueError("No datafiles provided")
    
    # Skipped file counter
    skipped_files = 0

    # ----- STRUCTURAL ANALYSIS -----
    for basename, files in datafiles:
                
        pipeline.source.load(files)
        n_frames = pipeline.source.num_frames
        final_frame = max(0, n_frames - 1)
        data = pipeline.compute(frame = final_frame)

        # File Name
        data_file_name = structural_analysis_dir / f"{basename}_{unique_file_identifier}.txt"

        # Structural Analysis File Existance-Checker
        data_exists = any(structural_analysis_dir.rglob(data_file_name.name))
        if data_exists and not REPLACE_OLD_FILES:
            skipped_files += 1
            continue 

        # Data
        specific_data = data_function(data)
        np.savetxt(data_file_name, specific_data, delimiter=",", fmt="%.6f")
        print(f"Created {basename}_{unique_file_identifier}.txt")

    # Print Skipped Files
    if skipped_files:
        print(f"Skipped {skipped_files} existing {unique_file_identifier} files")  

def ring_analysis(datafiles, pipeline, min_ring_size, max_ring_size, bond_length):
    
    # Create Bonds Modifier
    bond_modifier = CreateBondsModifier(cutoff=bond_length)
    pipeline.modifiers.append(bond_modifier)
    
    # Ring Analysis Modifier
    ring_mod = FindRingsModifier(minimum_ring_size=min_ring_size, maximum_ring_size=max_ring_size)
    pipeline.modifiers.append(ring_mod)

    # Analysis
    file_analysis(datafiles, pipeline, "ring", lambda data: data.tables["ring-size-histogram"].xy())

    # Remove Modifiers
    pipeline.modifiers.pop()
    pipeline.modifiers.pop()

def coordination_analysis(datafiles, pipeline, coordination_cutoff):
    
    # Coordination Analysis Modfier
    coord_mod = CoordinationAnalysisModifier(cutoff=coordination_cutoff)
    pipeline.modifiers.append(coord_mod)

    # Analysis
    file_analysis(datafiles, pipeline, "coordination", lambda data: data.particles['Coordination'])

    # Remove Modifier
    pipeline.modifiers.pop()
    
def energy_analysis(datafiles, pipeline):

    # No modifier required

    # Analysis
    file_analysis(datafiles, pipeline, "potential_energy", lambda data: data.particles["c_pea"])

def RDF_analysis(datafiles, pipeline, RDF_cutoff, bins):
    
    # Coordination Analysis Modfier for RDF
    RDF_coord_mod = CoordinationAnalysisModifier(cutoff=RDF_cutoff, number_of_bins=bins)
    pipeline.modifiers.append(RDF_coord_mod)

    # Analysis
    file_analysis(datafiles, pipeline, "RDF", lambda data: data.tables['coordination-rdf'].xy())

    # Remove Modifier
    pipeline.modifiers.pop()    

def bond_length_analysis(datafiles, pipeline, bins, bond_length, bond_length_analysis_cutoff):

    # Create Bonds Modifier
    bond_modifier = CreateBondsModifier(cutoff=bond_length)
    pipeline.modifiers.append(bond_modifier)

    # Bond Analysis Modifier
    bond_analysis_mod = BondAnalysisModifier(bins = bins, length_cutoff=bond_length_analysis_cutoff)
    pipeline.modifiers.append(bond_analysis_mod)

    # Analysis
    file_analysis(datafiles, pipeline, "bond_length", lambda data: data.tables["bond-length-distr"].xy())
  
    # Remove Modifiers
    pipeline.modifiers.pop()
    pipeline.modifiers.pop()
 

# ----- POSSIBLE ANALYSIS ------
# 1. Forces
# 2. Bond Angle
# 3. Conditional analysis (i.e. for sp/sp2/sp3 individually)
# 4. Young's Modulus
# 5. Coordination for each frame in a given sim plotted against simulation time
# ------------------------------

# -----------------------
# Use carefully - will regenerate ALL files (apart from renders)
REPLACE_OLD_FILES = True

if REPLACE_OLD_FILES:
    confirm = input("Are you sure you want to replace old files? (y/n): ").strip().lower()
    if confirm != "y":
        REPLACE_OLD_FILES = False
# -----------------------

bond_length_analysis(datafiles, pipeline, bins=1000, bond_length = 1.85, bond_length_analysis_cutoff=2.0)
RDF_analysis(datafiles, pipeline, RDF_cutoff=6.0, bins=200)
ring_analysis(datafiles, pipeline, min_ring_size=3, max_ring_size=24, bond_length=1.85)
coordination_analysis(datafiles, pipeline, coordination_cutoff=1.85)
energy_analysis(datafiles, pipeline)
#ovito_analysis(datafiles, pipeline)

470 LAMMPS simulation files imported
Skipped 470 existing bond_length files
Skipped 470 existing RDF files
Skipped 470 existing ring files
Skipped 470 existing coordination files
Skipped 470 existing potential_energy files


In [14]:
# ----- FILE ORGANISER -----
from pathlib import Path
import re

# Assign Directories
cwd = Path.cwd()
analysis_dir = cwd / "Analysis"
analysis_dir.mkdir(exist_ok=True)
ovito_dir = analysis_dir / "Ovito"
ovito_dir.mkdir(exist_ok=True)
structural_analysis_dir = analysis_dir / "Structural Analysis"
structural_analysis_dir.mkdir(exist_ok=True)

# Regex for capturing the MD and structural parameters
LAMMPS_regex_pattern = re.compile(
    r'^run_\d+_'                                 # captures run number
    r'(C_nvt_\d+atoms_[0-9]+(?:\.[0-9]+)?gcm_[0-9]+(?:\.[0-9]+)?minsep.data\_m\d+_c\d+)'  # structural and melt/quench parameters
    r'(?:[._].*|$)'                              # captures anything after the "key"
)

# Groups files with matching structural and melt/quench parameters 
def file_organiser(dir_path, OVERWRITE=False):
    
    for file in dir_path.iterdir():
        if not file.is_file():
            continue

        base = file.stem
        match = LAMMPS_regex_pattern.match(base)
        if not match:
            continue

        simulation_parameters = match.group(1)  # Matches simulation and structural parameters
        simulation_dir = dir_path / f"run_{simulation_parameters}"
        simulation_dir.mkdir(exist_ok=True)
        
        destination = simulation_dir / file.name

        if destination.exists():
            if OVERWRITE:
                print(f"Overwritten: {file.name}")
            else:
                print(f"Skipped Overwrite of {file.name}")
                continue
        
        # Move (and overwrite if allowed)
        file.replace(destination)

# -----------------------
# Use carefully - will replace ALL existing files 
OVERWRITE = True

if OVERWRITE:
    confirm = input("Are you sure you want to overwrite existing files? (y/n): ").strip().lower()
    if confirm != "y":
        REPLACE_OLD_FILES = False
# -----------------------

# Organise ovito files and structural_analysis files
file_organiser(ovito_dir, OVERWRITE=OVERWRITE)
file_organiser(structural_analysis_dir, OVERWRITE=OVERWRITE)


In [15]:
# ------ GRAPHICAL ANALYSIS --------

# Graphical data points are means of all repeat runs with errors given as 1 standard deviation
import pandas as pd
import numpy as np
import re

# Create Graphical Analysis Directories
from pathlib import Path

cwd = Path.cwd()
analysis_dir = cwd / "Analysis"
analysis_dir.mkdir(exist_ok=True)

graph_dir = analysis_dir / "Graphical Analysis"
graph_dir.mkdir(exist_ok=True)


# ------ FIGURE FORMATTING ------
import matplotlib.pyplot as plt
plt.style.use('1_column_fig.mplstyle')
# -------------------------------

# ------ DATA IMPORTING------
# analysis_type: density, quench, single_value 
# data_name: identifying name in data file name
# Simulation params melt_time, quench_time, density: int or None
# Returns a list of tuples (filepath, density, melt_time, quench_time)
def import_data_files(directory, data_name, analysis_type, density, melt_time, quench_time):
    
    directory = Path(directory)

    # Normalise analysis type to be non-case sensitive
    analysis_type_norm = str(analysis_type).lower()

    # Regex for data types
    data_file_regex = re.compile(fr".*({re.escape(data_name)}).*") #regex for finding coordination data
    density_regex = re.compile(r'_(\d+(?:\.\d+)?)\s*(?=gcm)') #regex for finding density data
    melt_quench_regex =  re.compile(r'm(\d+)_c(\d+)')  # regex for finding melt/quench parameters    

    imported_data_files = [] # in the tuple format: (filepath, density, melt, quench)

    run_folders = [d for d in directory.iterdir() if d.is_dir()] # list all run folders

    # Type checks for correct inputs
    try:
        melt_time_filter = int(melt_time) if melt_time is not None else None
    except (ValueError, TypeError):
        raise ValueError("melt_time must be an int or None")
    try:
        quench_time_filter = int(quench_time) if quench_time is not None else None
    except (ValueError, TypeError):
        raise ValueError("quench_time must be an int or None")
    try:
        density_filter = float(density) if density is not None else None
    except (ValueError, TypeError):
        raise ValueError("density must be a number or None")

    # Further Checks for correct for a given analysis
    if analysis_type_norm == "density":
        if melt_time_filter is None or quench_time_filter is None:
            raise ValueError("For density analysis, melt_time and quench_time must be provided (not None).")
        if density_filter is not None:
            raise ValueError("For density analysis, the 'density' parameter should be None (it's used as a filter for quench analysis).")
    elif analysis_type_norm == "quench":
        if melt_time_filter is None or density_filter is None:
            raise ValueError("For quench analysis, melt_time and density must be provided (not None).")
        if quench_time_filter is not None:
            raise ValueError("For quench analysis, the 'quench_time' parameter should be None (it's used as a filter for density analysis).")  
    elif analysis_type_norm == "single_value":
        if melt_time_filter is None or density_filter is None or quench_time_filter is None:
            raise ValueError("For single value analysis, all args must be specified (melt, quench, density)")
    
    if analysis_type_norm not in existing_analysis_types:
        raise ValueError(f"Unknown analysis_type: {analysis_type!r}")

    # Density analysis vs. Quench time analysis vs. variable density analysis
    for run_folder in run_folders:
        run_folder_name = run_folder.name
        
        # Exract density
        d = density_regex.search(run_folder_name)
        if d is None:
            continue                            
        try:
            file_density = float(d.group(1))   
        except ValueError:
            print(f"ERROR: Failed to extract {run_folder} density")
            continue

        # Extract melt/quench params
        m = melt_quench_regex.search(run_folder_name)
        if m is None:
            continue
        try:
            file_melt_time = int(m.group(1))    
            file_quench_time = int(m.group(2))  
        except ValueError:
            print(f"ERROR: Failed to extract {run_folder} melt/quench parameters")
            continue

        # List datafiles in each run_folder
        data_files = [f for f in run_folder.iterdir() if f.is_file()]
        
        for data_file in data_files:
            
            # Extract data file type
            ft = data_file_regex.search(data_file.name)
            if ft is None: 
                continue
            file_type = ft.group(1)

            if file_type == data_name: 
            
                # Density Analysis (returns files with matching melt and quench params)
                if analysis_type_norm == "density":
                    if (file_melt_time == melt_time_filter
                        and file_quench_time == quench_time_filter):
                        imported_data_files.append((data_file, file_density, file_melt_time, file_quench_time))

                # Quench Time Analysis (returns files with matching density and melt params)
                elif analysis_type_norm == "quench":
                    if (file_melt_time == melt_time_filter
                        and (abs(file_density - density_filter) < 1e-6)):
                        imported_data_files.append((data_file, file_density, file_melt_time, file_quench_time))

                # Single Density Analysis (returns files with matching density, melt and quench params)
                elif analysis_type_norm == "single_value":
                    if (file_melt_time == melt_time_filter
                        and file_quench_time == quench_time_filter
                        and (abs(file_density - density_filter) < 1e-6)):
                        imported_data_files.append((data_file, file_density, file_melt_time, file_quench_time))

    
    if not imported_data_files:
        print("No datafiles found for the current selection")
    else:
        num_imported_files = len(imported_data_files)
        print(f"{num_imported_files} matching data files found")
    return imported_data_files
# ----------------------------

# ------ PLOTTING DATA ------
# Plot_type: marker (with error bars), line (with shaded regions) 
# Note: only works with density or quench analysis
def plot(plot_type, data, analysis_type, x_label, y_label, chart_title, save_file_name):

    if plot_type not in existing_plot_types:
        raise ValueError(f"Unknown plot_type: {plot_type!r}")

    data = data.sort_values(f"{analysis_type}")
    fig, ax = plt.subplots()  # Local figure size for each plot
    
    x = data[f"{analysis_type}"].values
    mean = data["mean"].values
    std = data["std"].values
    
    if plot_type == "marker":
        ax.errorbar(x, mean, yerr=std,fmt='-o', capthick=0.5, elinewidth=0.5)

    elif plot_type == "line":
        alpha_fill = 0.25
        ax.plot(x, mean, label="Mean")
        ax.fill_between(x, mean - std, mean + std, alpha=alpha_fill)
                
    # Labels and Titles
    ax.set_xlabel(f"{x_label}")
    ax.set_ylabel(f'{y_label}')
    ax.set_title(f"{chart_title}")

    # Ensure directory exists
    graph_dir.mkdir(parents=True, exist_ok=True)

    filepath = graph_dir / save_file_name

    # Save Plot to Graphical Analysis
    plt.savefig(filepath)
    print(f"{filepath.name} created")
    plt.close(fig)  # Close figure to free memory
# ---------------------------
            
# Coordination analysis 
def coordination_analysis(analysis_type, directory, density, melt_time, quench_time, coordination_number):
    
    data_name = "coordination"

    # Label coordination number 
    if coordination_number == 2:
        y_label = "sp Carbon Proportion"
    elif coordination_number == 3:
        y_label = "sp2 Carbon Proportion"
    elif coordination_number == 4:
        y_label = "sp3 Carbon Proportion"
    else:
        print("ERROR: Coordination number should be between 2 and 4")
        y_label == f"{coordination_number} coordinate atoms"
        return

    if analysis_type == "density":
        
        datafiles = import_data_files(directory, data_name, analysis_type, None, melt_time, quench_time)

        if not datafiles:
            return None

        coordination_density_results = [] # In the form: (density, coordination_proportion)

        for file, density, _, _ in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                coordination_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
                        
            # Failsafe incase there's no data in the file
            if coordination_data.size == 0:
                continue
        
            # Calculate coordination proportions
            coordination_proportion = (np.count_nonzero(coordination_data == coordination_number) / coordination_data.size)
            coordination_density_results.append((density, coordination_proportion))
        

        x_label = "Density (g/cm³)"
        chart_title = f"Coordination vs. Density\nMelt Time = {melt_time} fs, Quench Time = {quench_time} fs"
        save_file_name = f"{y_label}_coordination_density_plot_m{melt_time}_c{quench_time}.png"


    elif analysis_type == "quench":
        
        datafiles = import_data_files(directory, data_name, analysis_type, density, melt_time, None)

        if not datafiles:
            return None

        coordination_density_results = [] # In the form: (density, coordination_proportion)

        for file, _, _, quench in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                coordination_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
                        
            # Failsafe incase there's no data in the file
            if coordination_data.size == 0:
                continue
        
            # Calculate coordination proportions
            coordination_proportion = (np.count_nonzero(coordination_data == coordination_number) / coordination_data.size)
            coordination_density_results.append((quench, coordination_proportion))
        
        x_label = "Quench Time (fs)"
        chart_title = f"Coordination vs. Quench Time\nMelt Time = {melt_time} fs  Density = {density} (g/cm³)"
        save_file_name = f"{y_label}_coordination_quench_plot_m{melt_time}_{density}gcm.png"
        

    df = pd.DataFrame(coordination_density_results, columns=[f"{analysis_type}", "coordination_proportions"])
    coordination_df = df.groupby(f"{analysis_type}")["coordination_proportions"].agg(["mean", "std"]).fillna(0.0).reset_index()  


    plot(plot_type = set_plot_type, data = coordination_df,
        analysis_type = analysis_type, x_label = x_label, y_label = y_label, 
        chart_title = chart_title,
        save_file_name = save_file_name)

# Ring Size analysis 
def ring_analysis(analysis_type, directory, density, melt_time, quench_time, ring_size):
    data_name = "ring"

    if analysis_type == "density":
        
        datafiles = import_data_files(directory, data_name, analysis_type, None, melt_time, quench_time)

        if not datafiles:
            return None

        ring_num_results = [] # In the form: (density, ring_num)

        for file, density, _, _ in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                ring_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
                        
            # Failsafe incase there's no data in the file
            if ring_data.size == 0:
                continue
                
            # Calculate number of rings
            num_rings = ring_data[ring_data[:, 0] == ring_size, 1][0]
            ring_num_results.append((density, num_rings))
        
        x_label = "Density (g/cm³)"
        chart_title = f"Number of {ring_size} Membered Rings vs. Density\nMelt Time = {melt_time} fs  Quench Time = {quench_time} fs"
        save_file_name = f"{ring_size}_ring_density_plot_m{melt_time}_c{quench_time}.png"

    elif analysis_type == "quench":
        
        datafiles = import_data_files(directory, data_name, analysis_type, density, melt_time, None)

        if not datafiles:
            return None

        ring_num_results = [] # In the form: (density, ring_num)

        for file, _, _, quench in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                ring_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
                        
            # Failsafe incase there's no data in the file
            if ring_data.size == 0:
                continue
                
            # Calculate number of rings
            num_rings = ring_data[ring_data[:, 0] == ring_size, 1][0]
            ring_num_results.append((quench, num_rings))
        
        x_label = "Quench Time (fs)"
        chart_title = f"Number of {ring_size} Membered Rings vs. Quench Time\nMelt Time = {melt_time} fs  Density = {density} (g/cm³)"
        save_file_name = f"{ring_size}_ring_quench_plot_m{melt_time}_{density}gcm.png"


    df = pd.DataFrame(ring_num_results, columns=[f"{analysis_type}", "num_rings"])
    coordination_df = df.groupby(f"{analysis_type}")["num_rings"].agg(["mean", "std"]).fillna(0.0).reset_index()  


    plot(plot_type = set_plot_type, data = coordination_df,
        analysis_type = analysis_type, x_label = x_label, y_label = f"{ring_size} Membered Rings", 
        chart_title = chart_title,
        save_file_name = save_file_name)

# Potential energy analysis 
def potential_energy_analysis(analysis_type, directory, density, melt_time, quench_time):

    data_name = "potential_energy"

    if analysis_type == "density":
        
        datafiles = import_data_files(directory, data_name, analysis_type, None, melt_time, quench_time)

        if not datafiles:
            return None

        PE_results = [] # In the form: (density, PE)

        for file, density, _, _ in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                energy_data = np.loadtxt(file)
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
            
            # Failsafe incase there's no data in the file
            if energy_data.size == 0:
                continue
                
            # Calculate number of rings
            mean_potential_energy = np.mean(energy_data)
            PE_results.append((density, mean_potential_energy))
        
        x_label = "Density (g/cm³)"
        chart_title = f"Mean Potential Energy vs. Density\nMelt Time = {melt_time} fs  Quench Time = {quench_time} fs"
        save_file_name = f"mean_potential_energy_density_plot_m{melt_time}_c{quench_time}.png"

    elif analysis_type == "quench":
        
        datafiles = import_data_files(directory, data_name, analysis_type, density, melt_time, None)

        if not datafiles:
            return None

        PE_results = [] # In the form: (density, PE)

        for file, _, _, quench in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                energy_data = np.loadtxt(file)
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
                
            # Fail-safe incase there's no data in the file
            if energy_data.size == 0:
                continue
                
            # Calculate mean potential energy
            mean_potential_energy = np.mean(energy_data)
            PE_results.append((quench, mean_potential_energy))
        
        x_label = "Quench Time (fs)"
        chart_title = f"Mean Potential Energy vs. Quench Time\nMelt Time = {melt_time} fs  Density = {density} (g/cm³)"
        save_file_name = f"mean_potential_energy_quench_plot_m{melt_time}_{density}gcm.png"


    df = pd.DataFrame(PE_results, columns=[f"{analysis_type}", "mean_potential_energy"])
    coordination_df = df.groupby(f"{analysis_type}")["mean_potential_energy"].agg(["mean", "std"]).fillna(0.0).reset_index()  


    plot(plot_type = set_plot_type, data = coordination_df,
        analysis_type = analysis_type, x_label = x_label, y_label = 'Mean Potential Energy (eV)', 
        chart_title = chart_title,
        save_file_name = save_file_name)

# Bond Length analysis
def bond_length_analysis(analysis_type, directory, density, melt_time, quench_time):

    data_name = "bond_length"

    if analysis_type == "density":
        
        datafiles = import_data_files(directory, data_name, analysis_type, None, melt_time, quench_time)

        if not datafiles:
            return None

        bond_length_results = [] # In the form: (density, mean_bond_length)

        for file, density, _, _ in datafiles: 
                        
            # Failsafe incase files don't load
            try:
                bond_length_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
            
            # Failsafe incase there's no data in the file
            if bond_length_data.size == 0:
                continue
        
            # Split columns
            x = bond_length_data[:, 0]   # bond length
            y = bond_length_data[:, 1]   # frequency

            # Calculate mean bond length w.r.t. density
            mean_bond_length = np.average(x, weights=y)
            bond_length_results.append((density, mean_bond_length))
        
        x_label = "Density (g/cm³)"
        chart_title = f'Mean Bond Length vs. Density\nMelt Time = {melt_time} fs, Quench Time = {quench_time} fs'
        save_file_name = f"bond_length_density_plot_m{melt_time}_c{quench_time}.png"

    elif analysis_type == "quench":
        
        datafiles = import_data_files(directory, data_name, analysis_type, density, melt_time, None)

        if not datafiles:
            return None

        bond_length_results = [] # In the form: (density, mean_bond_length)

        for file, _, _, quench in datafiles:

            # Failsafe incase files don't load
            try:
                bond_length_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
            
            # Failsafe incase there's no data in the file
            if bond_length_data.size == 0:
                continue
        
            # Split columns
            x = bond_length_data[:, 0]   # bond length
            y = bond_length_data[:, 1]   # frequency

            # Calculate mean bond length w.r.t. quench time
            mean_bond_length = np.average(x, weights=y)
            bond_length_results.append((quench, mean_bond_length))
        
        x_label = "Quench Time (fs)"
        chart_title = f"Mean Bond Length vs. Quench Time\nMelt Time = {melt_time} fs  Density = {density} (g/cm³)"
        save_file_name = f"bond_length_quench_plot_m{melt_time}_{density}gcm.png"


    df = pd.DataFrame(bond_length_results, columns=[f"{analysis_type}", "mean_bond_length"])
    bond_length_df = df.groupby(f"{analysis_type}")["mean_bond_length"].agg(["mean", "std"]).fillna(0.0).reset_index()  


    plot(plot_type = set_plot_type, data = bond_length_df,
        analysis_type = analysis_type, x_label = x_label, y_label = 'Mean Bond Length (Å)', 
        chart_title = chart_title,
        save_file_name = save_file_name)

# ------ RDF/DENSITY ANALYSIS ------
# Allows comparison of different densities
# The RDF is given as the mean of all repeats
# Shading represents the standard deviation
# Note: this assumes the same x values in the RDFS for the data, which is only true
# for the same number of atoms and bins --> specified in structure generator and earlier structural analysis
# for different analysis types it takes the first value of densities/quench_times if there is more than one
def RDF_density_analysis(directory, melt_time, quench_time, densities):
       
    data_name = "RDF"

    per_density_dataframes = {}

    for density in densities:
        datafiles = import_data_files(directory, data_name, "single_value", density, melt_time, quench_time)

        if not datafiles:
            continue

        RDF_results = [] # in the form (density, r_array, g_r_array)

        for file, file_density, _, _ in datafiles:
            # Failsafe incase files don't load
            try:
                RDF_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
            
            # Failsafe incase there's no data in the file
            if RDF_data.size == 0:
                continue
            
            r_array = RDF_data[:, 0]
            g_r_array = RDF_data[:, 1]

            RDF_results.append((file_density, r_array, g_r_array))
            
        # skip if no files found 
        if len(RDF_results) == 0:
            continue

        # Check that the r scales are the same across all repeats
        ref_r = RDF_results[0][1] #first r scale
        consistent_r = True
        for _, r_arr, _ in RDF_results[1:]:
            if not np.allclose(ref_r, r_arr, atol=1e-6):
                print(f"ERROR: RDF data not consistent in r scale — skipping density {density}")
                consistent_r = False
                break
        if not consistent_r:
            continue  # skip this density entirely

        # Expand each tuple (density, r_array, g_r_array) into rows to allow for aggregation
        rows = []
        for d, r_arr, g_arr in RDF_results:
            for r_val, g_val in zip(r_arr, g_arr):
                rows.append((d, r_val, g_val))
        
        # Dataframe
        df = pd.DataFrame(rows, columns=["density", "r", "g_r"])
        RDF_df = (df.groupby(["density", "r"])["g_r"].agg(["mean", "std"]).fillna(0.0).reset_index())
    
        # Saved to the dictionary
        per_density_dataframes[density] = RDF_df

    if not per_density_dataframes:
        print("ERROR: No data found for RDF plot")
        return

    # Plot Data
    # Shaded areas given by +- 1 standard deviation
    def plot_RDFs(rdf_dict, title, alpha_fill=0.25):
        
        fig, ax = plt.subplots()
        ax.set_xlabel("r (Å)")
        ax.set_ylabel("g(r)")
        ax.set_title(f"{title}")
        
        # sort densities for consistent legend order
        densities_sorted = sorted(rdf_dict.keys())

        for density in densities_sorted:
            df = rdf_dict[density]
            r = df['r'].values
            mean = df['mean'].values
            std = df['std'].values

            ax.plot(r, mean, label=f"{density:.2f} g/cm³")
            ax.fill_between(r, mean - std, mean + std, alpha=alpha_fill)

        ax.legend(title="Density")

        # Save image
        dens_str = "-".join(f"{d:.2f}" for d in densities)
        save_file_name = f"RDF_m{melt_time}_c{quench_time}_{dens_str}gcm.png"
        filepath = graph_dir / save_file_name
        plt.savefig(filepath)
        plt.close(fig)
        print(f"{filepath.name} created")

    plot_RDFs(per_density_dataframes, title=f"Radial Distribution Function\nMelt Time = {melt_time} fs, Quench Time = {quench_time} fs")
#-----------------------------------

# ------ RING SIZE/ DENSITY HISTOGRAM ------
# Allows comparison of different densities
# Shading represents the standard deviation
def ring_histogram_density_analysis(directory, melt_time, quench_time, densities):
    
    data_name = "ring"

    per_density_dataframes = {}

    for density in densities:
        datafiles = import_data_files(directory, data_name, "single_value", density, melt_time, quench_time)

        if not datafiles:
            return None

        ring_results = [] # in the form (density, ring_size_array, frequency_array) 

        for file, file_density, _, _ in datafiles:
                    
            # Failsafe incase files don't load
            try:
                ring_data = np.loadtxt(file, delimiter=',')
            except Exception as e:
                # skip bad files
                print(f"Skipping {file}: {e}")
                continue
            
            # Failsafe incase there's no data in the file
            if ring_data.size == 0:
                continue
            
            ring_size_array = ring_data[:,0]
            frequency_array = ring_data[:,1]

            ring_results.append((file_density, ring_size_array, frequency_array))
            
        # skip if no files found 
        if len(ring_results) == 0:
            continue

        # Expand each tuple (density, ring_size_array, frequency_array) into rows to allow for aggregation
        rows = []
        for d, ring_size_arr, frequency_array in ring_results:
            for r_val, f_val in zip(ring_size_arr, frequency_array):
                rows.append((d, r_val, f_val))        

        # Dataframe
        df = pd.DataFrame(rows, columns=["density", "ring_size", "frequency"])
        ring_df = (df.groupby(["density", "ring_size"])["frequency"].agg(["mean", "std"]).fillna(0.0).reset_index())
        
        # Saved to the dictionary
        per_density_dataframes[density] = ring_df

    if not per_density_dataframes:
        print("ERROR: No data found for ring histogram plot")
        return

    # Plot Data
    # Shaded areas given by +- 1 standard deviation
    def plot_ring_hist(rdf_dict, title, alpha_fill=0.25):
        
        fig, ax = plt.subplots() 
        ax.set_xlabel("Ring Size")
        ax.set_ylabel("Frequency")
        ax.set_title(f"{title}")
        
        # sort densities for consistent legend order
        densities_sorted = sorted(rdf_dict.keys())

        for density in densities_sorted:
            df = rdf_dict[density]
            r = df['ring_size'].values
            mean = df['mean'].values
            std = df['std'].values

            ax.errorbar(r, mean, yerr=std,fmt='-o', capthick=0.5, elinewidth=0.5, label=f"{density} g/cm³")

            #ax.fill_between(r, mean - std, mean + std, alpha=alpha_fill)

        ax.legend(title="Density")

        # Save image
        dens_str = "-".join(f"{d:.2f}" for d in densities)
        save_file_name = f"ring_histogram_m{melt_time}_c{quench_time}_{dens_str}gcm.png"
        filepath = graph_dir / save_file_name
        plt.savefig(filepath)
        plt.close(fig)
        print(f"{filepath.name} created")

    plot_ring_hist(per_density_dataframes, title=f"Ring Size Histogram\nMelt Time = {melt_time} fs, Quench Time = {quench_time} fs")
# ------------------------------------------

# ------ ANALYSIS PARAMETERS ------
existing_analysis_types = ["density", "quench", "single_value"]
existing_plot_types = ["marker", "line"]

set_analysis_type = "density"  # NOTE: "single_value" analysis is only valid for multi-density plots - do not use here
set_plot_type = "line"

set_density = 2.5 # for quench analysis
set_melt_time = 5000
set_quench_time = 10000 # for density analysis
set_analysis_directory = "Analysis/Structural Analysis"

set_coordination_number = 4
set_ring_size = 6
# ---------------------------------

coordination_analysis(analysis_type = set_analysis_type, directory=set_analysis_directory, 
                      density = set_density, melt_time=set_melt_time, quench_time=set_quench_time, 
                      coordination_number=set_coordination_number)

bond_length_analysis(analysis_type = set_analysis_type, directory=set_analysis_directory, 
                     density = set_density, melt_time=set_melt_time, quench_time=set_quench_time)

potential_energy_analysis(analysis_type=set_analysis_type, directory=set_analysis_directory,
                           density = set_density, melt_time=set_melt_time, quench_time=set_quench_time)

ring_analysis(analysis_type = set_analysis_type, directory= set_analysis_directory, 
              density = set_density, melt_time=set_melt_time, quench_time=set_quench_time, 
              ring_size=set_ring_size)


#Multi density plots
ring_histogram_density_analysis('Analysis/Structural Analysis', 
                                melt_time=5000, quench_time=10000, densities=(2.0,3.0))

RDF_density_analysis('Analysis/Structural Analysis', 
                     melt_time=5000, quench_time=10000, densities=(2.0,3.5,1.5))

210 matching data files found
sp3 Carbon Proportion_coordination_density_plot_m5000_c10000.png created
210 matching data files found
bond_length_density_plot_m5000_c10000.png created
210 matching data files found
mean_potential_energy_density_plot_m5000_c10000.png created
210 matching data files found
6_ring_density_plot_m5000_c10000.png created
10 matching data files found
10 matching data files found
ring_histogram_m5000_c10000_2.00-3.00gcm.png created
10 matching data files found
10 matching data files found
10 matching data files found
RDF_m5000_c10000_2.00-3.50-1.50gcm.png created
